In [2]:
import os
import glob
import tifffile as tiff
from cellpose import denoise, models, io
from skimage import measure 
import pandas as pd

# For testing

In [2]:
image_input_file = r"G:/Alexis_data/Project/SpatialOmics_cellpose/results/cropped_image/cropped_square_com_27.tif"


In [3]:
# Read the image
img = tiff.imread(image_input_file)

# Print the shape of the image
print(f"Image shape: {img.shape}")

Image shape: (2, 1922, 1922)


In [35]:
# model set up
# denoise_model = denoise.DenoiseModel(gpu=True, model_type='cyto3',  diam_mean=31.0)
model = models.CellposeModel(gpu=True, 
                             pretrained_model='G:/Alexis_data/Project/SpatialOmics_cellpose/models/HILT_Alexis_1113',
                             # model_type = "cyto3", 
                             diam_mean=31.0
                             )

# cellpose denoise and segmentation procedure
print("input image shape is: ", img.shape)



# # denoise
# imgs_dn = denoise_model.eval(img, batch_size=4,
#                             channels=[2,1],
#                             normalize=True,
#                             rescale=None,
#                             diameter=31,
#                             do_3D=False,
#                             bsize=224)

# segmentation
masks, flows, styles = model.eval(
    x=img,  # Your input image array
    batch_size=8,
    diameter=31.0,
    channels=[2,1],
    do_3D=False,
    channel_axis = None,
    # anisotropy=2.0    # Adjust anisotropy if needed
)

print("output image shape is: ", masks.shape)

input image shape is:  (2, 1922, 1922)
output image shape is:  (1922, 1922)


In [36]:
from skimage import measure
props = measure.regionprops_table(masks, properties=[ 'centroid'])
props_df = pd.DataFrame(props)

In [37]:
import pandas as pd

props_df

,centroid-0,centroid-1
0,10.225352,35.652582
1,4.251309,420.481675
2,5.593886,482.668122
3,35.483108,351.917793
4,24.613435,426.770772
...,...,...
147,1882.632792,192.117848
148,1874.884688,214.374291
149,1909.469837,217.676366
150,1909.203883,130.737864


In [22]:
# Save the masks as TIFF files
io.save_masks(
    images=[img],
    masks=[masks],
    flows=[flows],
    file_names=['G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835od'],
    png=False,
    tif=True,
    channels=[0, 0]
)  

In [18]:
from cellpose import models, io


# Save the segmentation output
io.masks_flows_to_seg(
    images=[img],  # List containing the image
    masks=[masks],            # List containing the mask
    flows=[flows],            # List containing the flows
    file_names=['G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835od'],  # Base filename for saving
    diams=31.0,              # Diameter used in Cellpose

)

print("Segmentation output saved to 'segmented_output_seg.npy'")

Segmentation output saved to 'segmented_output_seg.npy'


# For loop

In [3]:
def denoise_and_segment_image(img, img_name, output_path,  cellpose_model):
    """
    Perform denoising and segmentation on a 3D image using Cellpose.

    Parameters:
    - img (numpy.ndarray): Input image array to process.
    - img_name (str): Name of the input image (for logging and saving purposes).
    - output_path (str): Base path to save the output.
    - cellpose_model: Pre-initialized Cellpose segmentation model.

    Returns:
    None
    """
    # Display input image information
    print(f"Processing file: {img_name}, shape: {img.shape}")


    # Perform segmentation
    masks, flows, styles = cellpose_model.eval(
        x=img,  # Your input image array
        batch_size=8,
        diameter=31.0,
        channels=[2,1],
        do_3D=False,
        channel_axis = None,
    )
    
    print("Mask Segmentation Down!")

    # Create and save properties DataFrame
    props = measure.regionprops_table(masks, properties=['centroid'])
    props_df = pd.DataFrame(props)
    
    # Rename the columns
    props_df = props_df.rename(columns={
        'centroid-0': 'centroid_y',
        'centroid-1': 'centroid_x'
    })

    # Print number of rows (number of detected regions)
    print(f"Number of detected regions in {img_name}: {len(props_df)}")
    num_cells = len(props_df)

    # Create props CSV filename based on the original image name
    centroids_path = output_path + '_centroids.csv'

    props_df.to_csv(centroids_path, index=False)
    print(f"Properties saved to: {centroids_path}")

    # Save the masks as TIFF files
    io.save_masks(
        images=[img],
        masks=[masks],
        flows=[flows],
        file_names=[output_path],
        png=False,
        tif=True,
        channels=[0, 0]
    )

    io.masks_flows_to_seg(
        images=[img],  # List containing the image
        masks=[masks],            # List containing the mask
        flows=[flows],            # List containing the flows
        file_names=[output_path],  # Base filename for saving
        diams=31.0,              # Diameter used in Cellpose

    )

    print("Segmentation output saved to 'segmented_output_seg.npy'")

    return num_cells

In [4]:
def process_images_in_folder(input_folder, output_folder,  cellpose_model):
    """
    Process all TIF images in a folder: denoise, segment, and save the output.

    Parameters:
    - input_folder (str): Path to the folder containing input images.
    - output_folder (str): Path to save the processed output images.
    - denoise_model: Pre-initialized denoising model.
    - cellpose_model: Pre-initialized Cellpose segmentation model.

    Returns:
    None
    """
    i = 0

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Find all TIF files in the input folder
    file_paths = glob.glob(os.path.join(input_folder, "*.tif"))
    
    total_cells = 0
    for file_path in file_paths:
        # Read the image
        # print("file_path:", file_path)
        img = tiff.imread(file_path)

        # Get the base file name
        file_name = os.path.basename(file_path)
        # print("file_name:", file_name)

        file_base = os.path.splitext(file_name)[0]
        # print("file_base:", file_base)

        file_base = "cp_masks_" + file_base
        # print("file_base2:", file_base)

        # Define the output path for the current image
        output_path = os.path.join(output_folder, file_base)
        # print("output_path:", output_path)

        # Print the file name
        # print(f"Processing image {i}")
        
        img_name = file_name
        # Run the denoise and segmentation function
        num_cells = denoise_and_segment_image(
            img=img,
            img_name=file_name,
            output_path=output_path,   
            cellpose_model=cellpose_model
        )
        total_cells += num_cells
        i += 1

        print(f"Output of [{img_name}] saved as: {output_path}\n")

    return total_cells

In [5]:
# model set up
# denoise_model = denoise.DenoiseModel(gpu=True, model_type='cyto3',  diam_mean=31.0)
model = models.CellposeModel(gpu=True, 
                             pretrained_model='G:/Alexis_data/Project/SpatialOmics_cellpose/models/HILT_Alexis_250215_intest' ,
                             diam_mean=31.0
                             )

In [8]:

# Example usage
total = process_images_in_folder(
    input_folder='G:/Alexis_data/Project/SpatialOmics_cellpose/results/cropped_image/1835os',
    output_folder='G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os',
    cellpose_model=model
)

print(f"Total cells detected: {total}")

Processing file: cropped_square_com_1.tif, shape: (2, 2070, 2070)
Mask Segmentation Down!
Number of detected regions in cropped_square_com_1.tif: 243
Properties saved to: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_1_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_1.tif] saved as: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_1

Processing file: cropped_square_com_10.tif, shape: (2, 2070, 2070)
Mask Segmentation Down!
Number of detected regions in cropped_square_com_10.tif: 138
Properties saved to: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_10_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_10.tif] saved as: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_10

P

no masks found, will not save PNG or outlines


Mask Segmentation Down!
Number of detected regions in cropped_square_com_13.tif: 0
Properties saved to: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_13_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_13.tif] saved as: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_13

Processing file: cropped_square_com_14.tif, shape: (2, 2070, 2070)
Mask Segmentation Down!
Number of detected regions in cropped_square_com_14.tif: 10
Properties saved to: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_14_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_14.tif] saved as: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_14

Processing file: cropped_square_com_15.tif, shape: (2, 2070, 2070)

no masks found, will not save PNG or outlines


Mask Segmentation Down!
Number of detected regions in cropped_square_com_23.tif: 0
Properties saved to: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_23_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_23.tif] saved as: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_23

Processing file: cropped_square_com_3.tif, shape: (2, 2070, 2070)
Mask Segmentation Down!
Number of detected regions in cropped_square_com_3.tif: 149
Properties saved to: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_3_centroids.csv
Segmentation output saved to 'segmented_output_seg.npy'
Output of [cropped_square_com_3.tif] saved as: G:/Alexis_data/Project/SpatialOmics_cellpose/results/segmenatation/1835os\cp_masks_cropped_square_com_3

Processing file: cropped_square_com_4.tif, shape: (2, 1634, 2070)
Mask